# Adding Post-Newtonian general relativity corrections

It's easy to add post-newtonian corrections to your REBOUND simulations with REBOUNDx.  Let's start with a simulation without GR:

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.) # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630) # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame

sim.integrate(10.)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)

pomega = 0.0000000000000000


As expected, the pericenter did not move at all.  Now let's add GR

In [4]:
import reboundx
rebx = reboundx.Extras(sim)
print(rebx.sim.contents.G)
rebx.add_gr()

1.0


and run for another 100 time units

In [4]:
deltat = 100.
sim.integrate(sim.t + deltat)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)
juliancentury = 628.33195 # in yr/2pi
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.calculate_orbits()[0].pomega/deltat*juliancentury/arcsec))

pomega = 0.0000332618551934
Rate of change of pomega = 43.1083 [arcsec / Julian century]


As expected there was pericenter precession. The literature value is 42.98 arcsec / century. 

**Units**

To add GR, you need to pass `add_gr` the speed of light `c` in the units appropriate to the simulation.  By not passing a value above, `c` defaulted to our default units of AU, $M_\odot$ and yr/$2\pi$, for which `G=1` which is what we used for our initial conditions above.

But imagine now we wanted to instead use SI units:

In [5]:
import rebound
import reboundx
sim = rebound.Simulation()
sim.G = 6.67408e-11
sim.add(m=1.989e30) # Sun
sim.add(m=3.30216458e23,a=5.79090366e10,e=0.205630) # SI units
sim.move_to_com() # Moves to the center of momentum frame

import numpy as np
sim.integrate(10./2/np.pi)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)
rebx = reboundx.Extras(sim)
rebx.add_gr(c=3.e8)
deltat = 100./2/np.pi
sim.integrate(sim.t + deltat)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.calculate_orbits()[0].pomega/deltat*100./arcsec))

pomega = 0.0000000000000000
pomega = 0.0000000000000000
Rate of change of pomega = 0.0000 [arcsec / Julian century]


In [ ]:
print(rebx.gr.c)